<a href="https://colab.research.google.com/github/Prakhar-porwal/local-voice-tts-clean/blob/main/notebooks/fast_tts_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Local Voice TTS - Turbo Mode (Google Colab)

This notebook runs the **Local Voice TTS Backend** on a **Free Google Colab T4 GPU**.
This is roughly **10x-20x faster** than the Hugging Face CPU Free Tier.

### Instructions:
1.  **Runtime** -> **Change runtime type** -> **T4 GPU** (should be default).
2.  **Runtime** -> **Run all**.
3.  Wait for the **Public URL** (ending in `ngrok-free.app` or similar) to appear at the bottom.
4.  Copy that URL and use it in your Vercel Frontend!

In [ ]:
# @title 1. Setup & Install
# Clone the repository
!git clone https://github.com/Prakhar-porwal/local-voice-tts-clean
%cd local-voice-tts-clean

# Install system dependencies
!apt-get -y install espeak-ng libsndfile1

# Install Python dependencies
# 1. Remove torch (Colab has it)
# 2. Remove TTS (we install from source to fix Python 3.12 compatibility)
!sed -i '/torch/d' requirements.txt
!sed -i '/TTS/d' requirements.txt
!pip install -r requirements.txt
!pip install pyngrok nest_asyncio uvicorn

# Install TTS from source (bypasses PyPI version check for Py3.12)
!pip install git+https://github.com/coqui-ai/TTS

In [ ]:
# @title 2. Start the Turbo Backend
import os
import threading
from pyngrok import ngrok
import uvicorn
import nest_asyncio

nest_asyncio.apply()

# --- SET YOUR NGROK TOKEN IF YOU HAVE ONE ---

# ngrok.set_auth_token("YOUR_TOKEN_HERE")

# Set up the tunnel
try:
  public_url = ngrok.connect(7860).public_url
except Exception as e:
  print(f"Ngrok error (you might need to restart runtime if 'already online'): {e}")
  # Fallback to listing tunnels if they exist
  tunnels = ngrok.get_tunnels()
  if tunnels:
    public_url = tunnels[0].public_url

print("\n================================================================")
print(f"🚀 TURBO API URL: {public_url}")
print("================================================================\n")
print("Copy the URL above and paste it into your App settings or .env!")

# Run the Server
# We force CUDA (GPU) if available
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
uvicorn.run("main:app", host="127.0.0.1", port=7860)